# Basics of NLP

In this notebook, we will review some of the most fundamental ideas in Natural Language Processing (NLP).
We will start with a very naive kind of sentiment analysis and this will force us to also consider some of the most important
techniques for preprocessing natural language data.

## Sentiment analysis

One of the most popular kinds of NLP analyses used extensively in computational social science is sentiment analysis. The notion of sentiment refers to the emotional valence of a given text or utterance. Valence is typically defined in this context as a one-dimensional and bipolar construct and refers to the extent to which something is infused with negative or positive emotions/attitudes. However, there are also more elaborate kinds of sentiment analyses that define sentiment in terms of multiple dimensions usually referring to some model of basic (primitive) emotions (i.e. anger, disgust, fear, happiness, sadness, and surprise).

Sentiment analysis is often used in analyses of social media, speeches of public persons, press and many as well as in many other contexts.

In technical terms sentiment analysis may be conducted in many different ways (most sophisticated approaches are based on complex deep learning models). Nonetheless, all the methods are always based on some prior linguistic datasets (usually called lexicons or corpora) that assign some scores, representing emotional valences, to given words or phrases.

Here we will implement perhaps the most basic and naive (and frankly not very useful) sentiment analysis based on the so-called [AFINN](http://corpustext.com/reference/sentiment_afinn.html) lexicon. The lexicon assigns sentiment scores ranging from -5 (very negative) to +5 (very positive) to individual English words (and it includes only about 2500 of them). We will compute sentiment in a very simple way by just matching individual words from a larger text with their scores in the AFINN lexicon and computing different kinds of average scores based on that.

Later on, we will use a more nuanced type of sentiment tailored for web data (such as blog posts, etc.) called [VADER](https://www.aaai.org/ocs/index.php/ICWSM/ICWSM14/paper/viewPaper/8109), which is implemented in a powerful Python package for NLP called [NLTK](https://www.nltk.org/).

We start with a simple and naive sentiment to understand better the core idea of sentiment analysis. Moreover, it will allow force us to conduct a basic kind of data preprocessing typical for NLP studies.

## Data preprocessing

One of the biggest difficulties of NLP analysis stems from the fact that natural language is very contextual and messy. Many words may mean different things depending on context or might be spelled or written differently, for instance, depending on their position in a sentence while still being semantically equivalent. Other words may not have any intrinsic meaning as they play an only grammatical role. A good example of that are articles in English (i.e. a, an, the).

### Stop words

In our simple sentiment analysis, we will be concerned mostly with average sentiment scores over all words in a given text. Therefore, one of our concerns will be to first get rid of words with no clear semantics such as articles. Usually, such words in the context of NLP are called **stop words**. So we will want to get rid of all the stop words from our texts because they would bias our sentiment scores downwards.

### Tokenization

However, first, we have to notice that our approach will be based on the analysis of individual words. And initially, our texts will be single strings. Thus, first, we will have to decompose texts into single words. Such a process is usually called **tokenization** and it refers to a decomposition of a text into lower-order elements such as words or sentences. The naive way to do that would be to split a text by any kind of whitespace, but in practice, this is too simplistic. Luckily, people already studied this problem quite extensively and figured out better solutions, so we will not have to reinvent the wheel. Instead, we will use one of the methods offered by the NLTK package.

### Lemmatization

After tokenization, we would be able to remove stop words and lookup sentiment scores for the rest of the words. However, there is still some problem that we should address before that. Many words with the same meaning may be written differently in different the context, for instance, depending on whether they occur in singular or plural form or depending on tense, etc.

One way to deal with that is to convert words to their lemmas in the process called **lemmatization**. A lemma of a word is its core form. Below we provide sum examples:

* houses $\rightarrow$ house
* are $\rightarrow$ is
* mice $\rightarrow$ mouse
* becoming $\rightarrow$ become

Very accurate lemmatization is possible, but in general, it is rather hard and requires some additional work to be done that is beyond the scope of this course. Here we will do only a very simple kind of lemmatization that will allow us to simplify all plural forms into singular forms.

### Pipeline

Summing up, the data processing pipeline that we will use here will be the following:

$$\text{text} \rightarrow \text{tokenization} \rightarrow \text{lemmatization} \rightarrow \text{stop words removal} \rightarrow \text{sentiment analysis}$$

## Natural Language Toolkit (NLTK)

NLTK is one of the most important and popular Python packages for Natural Language Processing. It is a very powerful but also complex package and we will not discuss any details of how it works. Instead, we will only use a few tools it provides. However, what we will show is enough to conduct simple sentiment analysis. Thus, the techniques presented here will constitute the last element that together with the things we learned previously will allow you to conduct a simple computational study starting with data extraction and ending with simple natural language analysis.

As we already mentioned, one of the characteristic features of NLP is that it is usually based (one way or another) on some preexisting datasets called lexicons and/or corpora compile by linguists and other people who study natural languages. As a result, quite often working with NLTK starts with downloading some additional datasets that will be needed to perform particular analyses. Luckily, this is very easy with NLTK as it provides a very simple API for downloading missing datasets.

In [ ]:
# Import main packages
# --------------------
# We will use requests to download the AFINN lexicon.
# We will use the core module of NLTK to download missing datasets.
# --------------------
import requests
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
# Download lexicons etc.
nltk.download('punkt')
nltk.download('stopwords')
nltk.download("wordnet")

In [ ]:
# Text 1. | https://www.politico.eu/article/afghanistan-united-states-drugs-forces-the-secret-story-of-how-america-lost-the-drug-war-with-the-taliban/ 
text1 = """
As Afghanistan edged ever closer to becoming a narco-state five years ago, a team of veteran U.S. officials in Kabul presented the Obama administration with a detailed plan to use U.S. courts to prosecute the Taliban commanders and allied drug lords who supplied more than 90 percent of the world’s heroin — including a growing amount fueling the nascent opioid crisis in the United States. The plan, according to its authors, was both a way of halting the ruinous spread of narcotics around the world and a new — and urgent — approach to confronting ongoing frustrations with the Taliban, whose drug profits were financing the growing insurgency and killing American troops. But the Obama administration’s deputy chief of mission in Kabul, citing political concerns, ordered the plan to be shelved, according to a POLITICO investigation. Now, its authors — Drug Enforcement Administration agents and Justice Department legal advisers at the time — are expressing anger over the decision, and hope that the Trump administration, which has followed a path similar to former President Barack Obama’s in Afghanistan, will eventually adopt the plan as part of its evolving strategy. “This was the most effective and sustainable tool we had for disrupting and dismantling Afghan drug trafficking organizations and separating them from the Taliban,” said Michael Marsac, the main architect of the plan as the DEA’s regional director for South West Asia at the time. “But it lies dormant, buried in an obscure file room, all but forgotten.” A senior Afghan security official, M. Ashraf Haidari, also expressed anger at the Obama administration when told about how the U.S. effort to indict Taliban narcotics kingpins was stopped dead in its tracks 16 months after it began. “It brought us almost to the breaking point, put our elections into a time of crisis, and then our economy almost collapsed,” Haidari said of the drug money funding the Taliban. “If that [operation] had continued, we wouldn’t have had this massive increase in production and cultivation as we do now.” Poppy cultivation, heroin production, terrorist attacks and territory controlled by the Taliban are now at or near record highs. President Ashraf Ghani said recently that Afghanistan’s military — and the government itself — would be in danger of imminent collapse, perhaps within days, if U.S. assistance stops. But while President Donald Trump has sharply criticized Obama’s approach in Afghanistan, his team is using a similar one, including a troop surge last year and possibly another, and, recently, a willingness to engage in peace talks with the Taliban. The top-secret legal document that forms the plan’s foundation remains locked away in a vault at the U.S. Embassy in Kabul, and would need to be updated to reflect the significant expansion of the Taliban-led insurgency, said retired DEA agent John Seaman, who helped draft it as a senior law enforcement adviser for the Justice Department in Kabul. But he said the organizational structure of the Taliban leadership has remained mostly the same. “We have the ability to take these folks out,” he said. “Here’s your road map, guys. All you need to do is dust it off and it’s ready to go.” The plan, code-named Operation Reciprocity, was modeled after a legal strategy that the Justice Department began using a decade earlier against the cocaine-funded leftist FARC guerrillas in Colombia, in concert with military and diplomatic efforts. The new operation’s goal was to haul 26 suspects from Afghanistan to the same New York courthouse where FARC leaders were prosecuted, turn them against each other and the broader insurgency, convict them on conspiracy charges and lock them away. In Afghanistan, though, there was exponentially more at stake in what had become America’s longest war — and the clock was ticking. By the time that plans for Operation Reciprocity reached fruition, in May 2013, the conflict had cost U.S. taxpayers at least $686 billion. More than 2,000 American soldiers had given their lives for it. And the Obama administration already had announced it would withdraw almost entirely by the following year. Like the Bush White House before it, it had concluded that neither its military force nor nuanced nation-building could uproot an insurgency that was financed by deeply entrenched criminal networks that also had corrupted the Afghan government to its core. “We looked at this as the best, if not the only way, of preventing Afghanistan from becoming a narco-state,” said Seaman, referring to the government’s term for a country whose economy is dependent on the illegal drug trade. He described Operation Reciprocity as a fast, cost-effective and proven way of crippling the insurgency — akin to severing its head from its body — before the U.S. handed over operations to the Afghan government. “Without it,” he said, “they didn’t have a chance.” The document — a 240-page draft prosecution memo and 700 pages of supporting evidence — was the result of 10 years of DEA investigations done in conjunction with U.S. and allied military forces, working with embassy legal advisers from the departments of Justice and State. In May 2013, it was endorsed by the top Justice Department official in Kabul, who recommended it be sent to DOJ’s specialized Terrorism and International Narcotics unit in Manhattan. After agents flew in from Kabul for a three-hour briefing, the unit enthusiastically accepted the case and assigned one of its best and most experienced prosecutors to spearhead it. “These are the most worthy of targets to pursue,” Assistant U.S. Attorney Adam Fee, who had successfully prosecuted some of the FARC cases, wrote in an email to Seaman. But before Fee could pack for his first trip to Afghanistan, Operation Reciprocity was shut down. Its demise was not instantaneous. But the most significant blow, by far, came on May 27, 2013, when the then-deputy chief of mission, Ambassador Tina Kaidanow, summoned Marsac and two top embassy officials supporting the plan to her office, and issued an immediate stand-down order. In an interview, Kaidanow — currently the State Department‘s principal deputy assistant secretary for political-military affairs — said she didn’t recall details of the meeting or the specifics of the plan. But she confirmed that she felt blindsided by such a politically sensitive and ambitious effort and the traction it had received at Justice. If she did issue such an order, she said, it was because she — as the administration’s “eyes” in Afghanistan — had concerns it would undermine the White House’s broader strategy in Afghanistan, including a drawdown that included the DEA as well as the military. And the White House’s overriding priority ahead of the drawdown, she told POLITICO, was to use all tools at its disposal “to try and find a way to promote lasting stability in Afghanistan,” with peace talks integral to that effort. “So the bottom line is it had to be factored into whatever else was going on,” she said of the Taliban indictment plan. “We look at that entire array of considerations and think, you know, does it make sense in the moment? Does it make sense later on? Does it makes sense at all?” Its authors counter that Operation Reciprocity was designed in accordance with that White House strategy, an assertion backed up by interviews with current and former officials familiar with it and a review of government documents and congressional records. The authors believe the real reason it was shut down was fears it would jeopardize the administration’s efforts to engage the Taliban in peace talks and still-secret prisoner swap negotiations involving U.S. Army Sgt. Bowe Bergdahl. They tried to revive the effort after Kaidanow transferred back to Washington that fall, but by then, they say, circumstances had changed and the project never gained traction again. Recently, Seaman came forward to say that he and his former colleagues had all but given up on Operation Reciprocity until they discovered that the Trump administration had established a special task force to review and resurrect Hezbollah drug trafficking cases after a POLITICO report disclosed that they were derailed by the Obama administration’s determination to secure a nuclear deal with Iran. The Afghanistan team members said there are striking parallels between their case and Project Cassandra, the DEA code name for the Hezbollah investigations, as well as nuclear trafficking cases disclosed in another POLITICO report as being derailed because of the Iran deal. Taken together, they said, the cases show a troubling pattern of thwarting international law-enforcement efforts to the overall detriment of U.S. national security. Now they are hoping the Trump administration will review and revive Operation Reciprocity, too, saying Trump’s Afghanistan strategy cannot succeed without also incorporating an international law enforcement effort targeting the drug trade that helps keep the Taliban in business. Besides helping the military take strategic leaders off the battlefield, they said, it could provide much-needed leverage to finally bring the militant group to the negotiating table and also break up the criminal patronage networks undermining the Kabul government. For now, though, the plan remains buried in DEA files, and even most agency leadership is unaware of it, several current and former agency officials said. “I don’t think a lot of people even know that we did this, that this plan is in existence and is a viable thing that can be resurrected and completed,” said Marsac, whose eight years in and around Afghanistan for the DEA make him one of the longest-serving Americans there during the war. Such an undertaking would involve serious logistical challenges to capture drug lords and prosecute them in the United States, not to mention the destabilizing effect on the Afghan economy, from farmers who grow poppies to corrupt government officials accustomed to bribes. “We’ve made a deal with the devil on many occasions, in an effort not to antagonize anybody and kick the can down the road,” Marsac said. “But you’ve got to cut that off. It might be painful at first, but it has to be confronted.” Haidari, the director-general of Policy and Strategy for Afghanistan’s Ministry of Foreign Affairs, agrees and says it is something his country cannot yet do entirely on its own. Haidari recently helped lead a summit meeting in Kabul of 23 countries, including the United States, in proposing another round of peace talks with the Taliban as well as more military aid. Last month, Afghanistan had its first official cease fire since the insurgency began, but it lasted only three days — and demands that the Taliban get out of the narcotics trafficking business weren’t among the conditions. Haidari said the missing ingredient in the current scenario is a robust U.S. law enforcement effort to help Afghanistan starve the insurgency by attacking the Taliban’s drug funding, which, he noted, was precisely what Operation Reciprocity was designed to do. “That much money automatically involves their leadership and shows that they are narco-terrorists. You have to go after them,” even if peace talks are also pursued, Haidari said. “If you want to make peace with them, and you discontinue going after them, then the DEA is no longer allowed to do what it needs to do. And that is exactly what happened.” 

The alliance of the kingpins
 Obama was upbeat in his June 2011 address announcing a gradual end to the U.S. war in Afghanistan, saying, ”We’re starting this drawdown from a position of strength.” The rugged country that once provided Al Qaeda its haven no longer represented the same terrorist threat to the American people, Obama said, and U.S. and coalition forces had thwarted the insurgency’s momentum. The DEA’s Marsac believed from his many years in country that the situation on the ground wasn’t nearly as stable as Obama suggested. And that things were getting worse, not better. Obama was correct that most of Al Qaeda’s remaining forces had left for neighboring Pakistan. But Taliban-controlled territory was now home to at least a dozen other terrorist groups with international aspirations. The Taliban itself had evolved, too, from an insular group without animus toward the United States into a lethal narco-terrorist army waging war against the American forces that had deposed it for its indirect role in the 9/11 attacks. To finance its insurgency, the Taliban was reaping anywhere from $100 million to $350 million a year from its cut of the narcotics trade in hashish, opium, heroin and morphine, according to U.S., United Nations and other estimates. Much of the money went to pay for weapons, explosives, soldiers for hire and bribes to corrupt government officials. For decades, much of the region’s narcotics trade had been controlled by the Quetta Alliance, a loose confederation of three powerful tribal clans living in the Pakistani border town of the same name. At a June 1998 summit, the clan leaders gathered secretly to approve another alliance — with the Taliban, which ruled Afghanistan at the time, according to classified U.S. intelligence cited in Operation Reciprocity legal documents. Under the “Sincere Agreement,” the drug lords pledged their financial support for the Taliban in exchange for protection of their vast swaths of poppy and cannabis fields, drug processing labs and storage facilities. The ties were solidified further when the U.S. invasion toppled the Taliban after 9/11 and forced top commanders to flee to Quetta, where they formed a shura, or leadership council. In the early years of the U.S. occupation, the Pentagon and CIA cultivated influential Afghan tribal leaders who were not part of the Quetta Alliance, even if they were deeply involved in drug trafficking, in order to turn them against the Taliban. That willingness to overlook drug trafficking was assisted by their belief that the drugs were going almost entirely to Asia and Europe. But a lot of Afghan heroin was also coming into the United States, indirectly, including through Canada and Mexico, according to DEA, Justice Department and congressional officials and documents. Over time, growing numbers of Americans addicted to legally prescribed opioids were finding an alternative in the ample, but often deadly, narcotics supply on the streets. Even as the body counts mounted in Afghanistan, few Americans associated the war with growing opioid death and addiction rates in the U.S., including, importantly, appropriators in Congress. Lawmakers spent billions of taxpayer dollars annually on both the U.S. military campaign and reconstruction effort. But they earmarked just a tiny percentage of that for DEA efforts to counter the drug networks bankrolling the increasingly destructive attacks on both of them, records and interviews show. As a result, as of 2003, the DEA deployed no more than 10 agents, two intelligence analysts and one support staff member in the entire country. The agency’s primary mission was to disrupt and dismantle the most significant drug trafficking organizations posing a threat to the United States. Another mission was to train Afghan authorities in the nuts and bolts of counternarcotics work so that they could take on the drug networks themselves. Over the next three years, as the U.S. military cut back its presence in Afghanistan to focus on the Iraq War, the Taliban roared back to life. The DEA agents and their Afghan protégés were left to stanch the flow of drug money to the growing insurgency. Even after the U.S. and NATO countries began adding troops in 2006, the Afghan police and military counternarcotics forces were outgunned, outnumbered and outspent by the drug traffickers and their Taliban protectors, according to documents and interviews. Kabul’s criminal justice system remained a work in progress. Afghan prosecutors, with help from the DEA and the Justice Department, were putting away 90 percent of those charged with narcotics crimes. But most were two-bit drug runners whose convictions didn’t disrupt the flow of drug money, records show. Washington was coming to the realization that the Kabul government lacked the institutional capacity and the political will to take on the top drug lords, according to Rand Beers, who held a top anti-narcotics position in the George W. Bush administration. Lucrative bribes had compromised police and government officials from the precinct level to the inner circle of U.S.-backed President Hamid Karzai. That meant the more senior that suspected drug traffickers were, the less successful U.S. authorities were in pressuring the Afghans to act against them. As had been the case in Colombia, the drug kingpins were overseeing what had become vertically integrated international criminal conglomerates that generated billions of dollars in illicit annual proceeds. That made them, effectively, too big for their home government to confront. JOHN SEAMAN: Retired DEA agent and Justice Department contractor in Kabul who distilled mountains of U.S. and Afghan evidence into a 940-page prosecution plan that detailed a decade-long complex conspiracy case against Taliban leaders and drug lords, traffickers, money launderers and other alleged associates. The only criminal justice system willing and able to handle such networks was the one in the United States. By then, the U.S. Justice Department had indicted and prosecuted significant kingpins from Mexico, Thailand and, beginning in 2002, dozens of FARC commanders and drug lords from Colombia. In response, the DEA took two pages from its “Plan Colombia” playbook. It began embedding specially trained and equipped drug agents in military units, to start developing cases against the heads of the trafficking networks. It also worked closely with specially vetted Afghan counternarcotics agents. These Afghans were chosen by DEA agents for their courage, experience and incorruptibility, and then polygraphed and monitored to keep them honest. Together, the vetted Afghans and their DEA mentors established a countrywide network of informants and undercover operatives that penetrated deeply into the transnational syndicates. The crown jewel of that effort was a closely guarded electronic intercept program, in which DEA agents showed their Afghan counterparts how to obtain court-approved warrants and develop the technical skills needed to eavesdrop on communications. The hundreds of warrants authorized by Afghan judges provided a real-time window into the flows of drugs and money — from negotiations of individual narcotic sales to forensic road maps of the trafficking networks’ logistics and financial infrastructure. DEA agents also worked with a special Sensitive Investigative Unit to map the drug, terror and corruption networks. As the insurgency grew and became more costly to sustain, that evidence began to show the Taliban methodically assuming a more direct operational role in the drug trade, pushing out middlemen and extracting more profit — and money for the war effort — at every step of the process. All of the evidence was admissible in courts in Kabul and the United States. And it led agents straight to the top of the Taliban leadership — including its one-eyed supreme commander, Mullah Mohammad Omar, according to documents and interviews. By the end of the Bush administration, the Justice Department had indicted four top Afghan drug lords, who were ultimately captured and flown stateside or lured there under pretense, then prosecuted and convicted. A top-secret “target list” circulating at the time drew a bull's-eye on three dozen others who were next in the barrel, a Pentagon counternarcotics official said in an interview. As Bush prepared to pass the reins of government to Obama, it was clear to both administrations that the Afghan government wouldn’t be able to halt the flow of drug money to the insurpgency on its own. TINA KAIDANOW: State Department deputy chief of mission in Afghanistan who issued an immediate stand-down order halting Operation Reciprocity after discovering Justice Department prosecutors in New York had approved building a narcoterrorism criminal conspiracy case against Taliban leader Mullah Omar and 25 top associates. Bush’s outgoing Ambassador William Wood acknowledged as much in a withering cable back to Washington in January 2009, saying that the narcotics trade had become so pervasive that it made up one-third of Afghanistan’s gross domestic product, but “no major drug traffickers have been arrested and convicted [by local authorities] in Afghanistan since 2006.” The battle against the Taliban would have to extend to courtrooms in the United States. 

Launching ‘Plan Afghanistan’
 The incoming Obama administration also publicly backed the “kingpin” strategy, as part of a counterinsurgency plan that focused on increased interdiction and rural development. “Going after the big guys” was how Richard Holbrooke, Obama’s special representative for Afghanistan and Pakistan, described it to Congress. That March, the DEA announced the most ambitious overseas deployment surge in its 40-year history — a six-fold increase of agents from 13 to 81. Not everyone was an unflinching fan of the DEA’s approach. Many people in and out of the government feared that targeting those at the apex of the drug trade could backfire in a place like Afghanistan, where it often meant taking on tribal leaders with armies of fighters, tanks and even missiles at their disposal, recalled Kenneth Katzman, a senior analyst on Afghanistan issues for the Congressional Research Service, the independent research arm of Congress. “These guys are powerful people,” Katzman told POLITICO. “Many have militias, and there are tribes, and subtribes, that depend on them for sustenance. You try to arrest someone like [that] and you are going to have a rebellion on your hands.” MICHAEL MARSAC: Drug Enforcement Administration regional director in Kabul who launched Operation Reciprocity to combine 10 years of DEA investigations tying Taliban leaders directly to the global heroin trade into one unprecedented prosecution in U.S. courts before President Barack Obama withdrew American forces from Afghanistan. But Haidari — then Afghanistan’s top national security diplomat in Washington — hailed the shift as being not only urgently needed but long overdue. "A surge not only of military but law enforcement is exactly what we need,” said Haidari, who was then a senior official in the Karzai government. “It is something we have always demanded of the U.S. government.” The DEA agents answering the call included a former Denver Broncos linebacker-turned-wiretapping expert and a former Marine with a Harvard Law degree. Several veteran commandos of the agency’s Latin American drug wars signed up too, including one storied agent who had built the case against the FARC. “It was 'Game on,'” Marsac said. “People wanted in on the fight. We were pulling them in from everywhere, and bringing them over in waves.” After completing several months of special operations training, new agents hit the ground running, sweeping through fortified drug compounds as allied military forces provided cover fire. The agents seized and cataloged as evidence multi-ton caches of narcotics, as well as stockpiles of Taliban weapons found, increasingly, alongside them. In November 2009, three DEA agents and seven American soldiers were killed when their helicopter crashed after a particularly intense drug raid in western Afghanistan. Obama and his attorney general, Eric Holder, traveled to Dover Air Force Base to receive their bodies. For the tight-knit and fast-growing DEA team in country, the fight against the Taliban was, from that point on, an intensely personal one. So-called FAST teams — for foreign-deployed advisory support — brought Afghan drug agents to the front lines of the drug war, including Helmand Province, the epicenter of both the drug trade and the insurgency. U.S. and coalition military forces now embraced both the DEA agents and their Afghan trainees as full partners who were making significant inroads in attacking the increasingly intertwined drug-terror networks. M. ASHRAF HAIDARI: Afghan counternarcotics official who lobbied Bush, Obama and Trump officials — mostly unsuccessfully — for more aggressive law enforcement efforts to take out drug kingpins and to stanch the flow of illicit narcotics proceeds that have fueled the Taliban insurgency and corrupted the Kabul government. On June 22, 2011, Obama formally announced a September 2014 drawdown date for almost all U.S. troops and DEA agents. Marsac, the leader of all the DEA staff in the region, figured he had two years, at most, in which to marshal his agency’s newfound horsepower in ways that would make a lasting difference and give the Afghans a fighting chance on their own. There wasn’t time to wrap up the myriad open investigations, even the multiyear ones targeting the kingpins. So Marsac proposed a legal Hail Mary of sorts: one giant U.S. conspiracy prosecution of the trafficking chieftains and the Taliban associates they financed. In January 2012, he assembled a team to review the mountains of evidence in DEA vaults to see whether it supported such a prosecution. The Justice Department had used such “wheel conspiracy” prosecutions for decades against international organized crime syndicates and drug cartels that had many tentacles. One especially potent advantage of such an approach was that evidence gathered against each defendant could be used to strengthen the overall conspiracy case against all of them. The team concluded that the evidence didn’t support a conspiracy case centering on the fractious and fragmented trafficking networks. But Marsac believed it might support something even more audacious, which DEA and Justice had never done before: a conspiracy case combining major drug traffickers and terrorist leaders. The Taliban senior leadership would be the hub in the center of the wheel, and its various trafficking partners, money launderers and the Quetta Shura the spokes arrayed around it. The main charge: That the Taliban had engaged in a complex conspiracy to advance the war effort through the production, processing and trafficking of drugs. DAVID SCHWENDIMAN: Justice Department attache in Kabul who advised and supported Operation Reciprocity, fought unsuccessfully to save it and warned a DOJ lawyer in Washington that Kaidanow’s plan to dismantle DEA efforts in Afghanistan would leave “the Afghans blind” to the Taliban’s drug-fueled insurgency. Marsac obtained approval from DEA’s Special Operations Division, a multi-agency nerve center that coordinates complex international law enforcement efforts. His deputy later did the same with DEA’s New York field office, which would be needed to help with support and logistics, such as safeguarding evidence and shepherding Afghan witnesses in country to testify before the grand jury hearing testimony in the case. Marsac opened an official case file and, requiring a name, called it Operation Reciprocity. It would be DEA’s way of settling the score against the Taliban, he told the team, for its complicity in the 9/11 attacks and the deaths of the DEA’s own agents. 

A tapestry of criminality
 By the end of 2012, the team members were struggling to make progress on building the conspiracy case, given their crush of daily caseload demands. So Marsac asked the Justice Department attache in Kabul for reinforcements. Specifically, Marsac wanted John Seaman, his old partner from their early days in Denver, who had become one of the DEA’s top experts in building conspiracy cases. After retiring in 2005 and doing some contract police work in Iraq, Seaman had spent the previous year on a Justice Department contract helping the Afghans identify sensitive anti-corruption and drug cases to pursue. With time running out in Afghanistan, Marsac hoped Seaman could find a way to jump-start Operation Reciprocity. The Justice Department’s attache in Kabul, David Schwendiman, himself a veteran prosecutor of international war crimes tribunals, quickly approved the request. Marsac and Seaman believed the evidence for a Taliban-led conspiracy existed somewhere in the thousands of intercept recordings, cooperating witness statements, financial transaction records and everything else that DEA personnel had gathered, processed and filed away since first deploying in 2002. Seaman’s particular talent was in finding the puzzle pieces and understanding how they fit together. Seaman, who was then 60 years old and a cancer survivor, scoured the evidence with focused intensity. Marsac would often leave work around 10 p.m., he said, “and I’d come back in the morning and John would still be still there.” A few weeks later, Seaman quietly took Marsac aside and showed him a sheaf of papers summarizing evidence to build a prosecution against Taliban leaders and drug lords. “You’ve got it,” he said. “It’s there.” Marsac and Seaman set up a war room within the DEA’s bunker near the Kabul airport, far from the U.S. Embassy. They papered the walls with photos of suspects, maps, charts and to-do lists. STEPHEN MCFARLAND: State Department ambassador overseeing law enforcement and “rule of law” programs in Afghanistan whose support of DEA counternarcotics investigations into politically sensitive targets — including Operation Reciprocity — prompted clashes with Kaidanow and his early and forced transfer back to Washington. One by one, Seaman constructed “target memos” for 26 key conspirators, each memo featuring 30 or so pages of alleged crimes committed, witnesses and evidence, along with, most importantly, a list of investigative avenues to pursue. Seaman then weaved all of it into the 240-page prosecution memo, an extraordinarily detailed tapestry of Afghanistan’s narcotics trade and the Taliban’s central — and financial — role in it from 1990 onward. All the information in the memo was relayed to agents in the field, and to the legal advisers in Kabul. The team enlisted Ambassador Stephen McFarland, one of five State Department officials of ambassadorial rank in Afghanistan. McFarland’s special role was to oversee DEA and other law-enforcement programs. Schwendiman, the Justice attache, was encouraged. He sent for the FARC case files from Washington and determined that the Kabul investigators exceeded the standard of evidence used to indict and convict the Colombian guerrilla commanders in U.S. courts. On his recommendation, the team sent the prosecution memo to the Justice Department’s Terrorism and International Narcotics unit in the Southern District of New York, which was known for taking on, and winning, the most ambitious and complex conspiracy cases. Three Operation Reciprocity agents flew to Manhattan to brief the prosecutors, who quickly greenlighted taking the case. They spent three hours strategizing and discussing the monumental challenges inherent in building such a case, securing final DOJ headquarters approval and taking it to trial. The logistical hurdles would be predictable — and surmountable — but the political ones would not. All agreed, however, that with Afghanistan descending into chaos, they had to try, according to Michael Schaefer, the supervisory DEA agent leading the investigation, and a second meeting participant. Bringing Mullah Omar and at least some of his top commanders to New York was unlikely, given the military’s inability to kill them over the past decade, they figured. But handing up detail-rich indictments would have immediate, as well as long-term, benefits, they said. Exposing the Taliban leadership’s direct role in the narcotics trade would undermine its campaign to win political legitimacy internationally. Related sanctions would block its access to the global financial system. And international arrest warrants, Schaefer said, would “fence in” the younger indictment targets enjoying jet-setter lifestyles outside Afghanistan. The indictments also would empower authorities everywhere to attack the drug lords’ transport and distribution networks, which snaked through dozens of countries, including the United States. And, the agents and prosecutors concluded, the case would serve as a model for prosecuting other narco-terrorist organizations, especially those operating in places where military force isn’t an option, Schaefer recalls. Fee, the newly appointed prosecutor, had won convictions against FARC commanders, Al Qaeda terrorists who blew up two U.S. embassies in Africa and other international defendants. He knew how to manage the intractable conflicts arising in such cases with U.S. diplomatic, intelligence and military officials. So the DEA agents were especially buoyed by Fee’s assessment that the case was winnable. “He told us the evidence was so strong,” Schaefer said, “that I’m ready to put you guys in front of a grand jury right now.” Back in Kabul, DEA chief Marsac shared the news with McFarland, the State Department’s law enforcement coordinator. The 35-year veteran diplomat had been a forceful advocate for the DEA in Afghanistan, especially when Kaidanow opposed enforcement efforts she believed would upset influential Afghan constituencies, McFarland told POLITICO. Given that history, he said, he concluded that he had to tell her the news immediately, and that she wasn’t going to like it. “I knew I would be walking into the middle of a shit storm,” he said. “I just had no idea how big it would be.” 

Throwing a wrench into things
 Kaidanow was, indeed, enraged during the meeting in her office, and accused the three men she had summoned there of insubordination, they said. What’s more, she told them, they had undermined the Obama agenda in Afghanistan by proposing the plan and getting Justice Department buy-in without her knowledge or approval, Marsac and McFarland said. “She said you’re throwing a wrench into things,” Marsac said, “but in a lot stronger language than that.” Schwendiman declined comment for this article but gave a similar account in two confidential internal Justice Department memoranda obtained by POLITICO. Kaidanow also threatened to “curtail” the three officials, the State Department term for firing them from their embassy positions and sending them home, the men said. But first, she gave Marsac three minutes to explain why she shouldn’t shut down the operation immediately. He ticked off the reasons, including that any indictments were months away. They could be kept secret, or leveraged strategically to help, not hinder, U.S. negotiators during peace talks. The way Marsac saw it, the case would show that the U.S. was committed to the rule of law, by prosecuting narco-terrorists as criminals rather than killing them with military force. Also, U.S. intelligence showed that Taliban drug lords were so against peace talks that they were threatening, and killing, associates who even considered them. With the drawdown imminent, he believed, Taliban shot-callers were just feigning interest in negotiations to burnish their political credentials and run out the clock. Before Marsac could finish, he said, Kaidanow cut him off, ordering him to stand down the operation and retrieve all documentation about it from the Justice Department. The DEA and Justice officials didn’t know it at the time, but they were proposing to indict top Taliban leaders at the exact moment the administration was secretly finalizing the launch of peace talks with the organization, set for Qatar the next month. Secretary of State John Kerry was arriving even sooner to soothe President Karzai, who was furious that the White House had excluded his government from the talks, POLITICO has learned. And clandestine negotiations were well underway already with the Taliban to get back Bergdahl, the Army sergeant who had deserted his post four years earlier, several current and former U.S. officials confirmed. The way Kaidanow saw it, the problem wasn’t just one of spectacularly bad timing. Pitching the plan to Justice was part of a pattern in which the DEA team — and its supporters at the embassy — had defied her “chief of mission authority,” or her role as coordinator of all U.S. agencies in the country in order to accomplish the administration agenda. “We were trying very hard to keep Afghanistan on a course, and there was no single way to address the Taliban problem,” Kaidanow said, adding that her interest in putting the brakes on any DEA efforts in country would have been driven by concerns that “everything they did was well vetted and coordinated” with other U.S. efforts. Kaidanow declined to comment on the three men’s accounts of the meeting, but denied threatening to dismiss anyone. But, she said, “In my career I have had to make many tough decisions that have not pleased everyone, but, as deputy chief of mission, I was responsible for implementing the policy set by the administration, and I stand by those decisions.” In interviews, the DEA and Justice officials countered that while Kaidanow did coordinate the overall U.S. mission in Afghanistan, they also were deployed there to represent the best interests of their home agencies internationally. As such, they reported, ultimately, to top DEA and Justice officials in Washington, who would decide whether to approve any Taliban indictments in consultation with others in the interagency process. Often, they said, they weren’t allowed to discuss such investigations with embassy leadership due to concerns about well-connected targets being tipped off. After the meeting with Kaidanow, Marsac went straight to his office and drafted a formal appeal. Instead of sending it, though, he and Schwendiman opted to lay low and launch their appeal after Kaidanow returned to Washington in the fall. Schwendiman enlisted McFarland in the plan, saying in a July 3 letter that they should lay the groundwork “as soon as we can.” “But there is no reason to think coming back to it in six months when cooler heads prevail will damage [its] prospects,” Schwendiman wrote. Waiting, however, turned out to be a fatal mistake. Nine days after Schwendiman’s letter, McFarland was sent back to Washington. In an effort to keep his job, McFarland had formally apologized to Kaidanow, assured her that the case files had been retrieved from Justice, and said he had much to contribute during what would certainly be a difficult handover of law enforcement affairs. “I said a procedural mistake was made here, we corrected it and no harm was done,” McFarland told POLITICO. Kaidanow denied forcing him out, but he maintained that she played a role in doing so. McFarland said her response was “totally disproportionate” and served to torpedo his chances of securing another ambassadorship. He left the State Department not long after returning to Washington. But McFarland has no regrets about supporting the Taliban indictment plan, which he said was totally in line with what the DEA was supposed to be doing in Afghanistan. “If you’re out there killing Americans,” he said, “it’s fair for us to go after you, not just with guns but to go after your [drug] money too.” 

Leaving the Afghans blind
 The Operation Reciprocity team unraveled quickly after that. Marsac himself left in August, citing a long-overdue transfer to DEA headquarters to be closer to his wife and daughter. He too took early retirement soon after that. Also in August, Schwendiman wrote a secret memorandum warning a senior Justice official in Washington that Kaidanow had iced Operation Reciprocity and ousted McFarland as part of a broader campaign to immediately shut down all U.S. law enforcement operations in Afghanistan — especially the DEA’s — even though the drawdown was more than a year away. Without McFarland to protect those operations, Kaidanow was going so far as to block DEA agents from investigating cases with their Afghan counterparts, the Justice attache wrote in the Aug. 22, 2013, “For Your Eyes Only” memo. And she was eliminating so many DEA positions in country, Schwendiman wrote, that it would be “problematic if not impossible” for Afghanistan to maintain the intelligence-gathering programs that formed the backbone of its entire counternarcotics and anti-corruption effort. That would undermine U.S. security interests in Afghanistan, wrote Schwendiman. “The Afghans, of course, will be left blind if this happens,” he added. In December, Schwendiman retired from the Justice Department and soon took a top job in Kabul for SIGAR, the independent U.S. watchdog agency known formally as the Special Inspector General for Afghanistan Reconstruction. Seaman also cut short his Justice Department contract, and returned stateside. Schaefer had some success in obtaining embassy buy-in after Kaidanow left. But by early 2014, some influential new DEA management officials were not supportive, saying the evidence for such a wide-ranging conspiracy wasn’t there, that they lacked the bandwidth to develop it or that they favored some individual prosecutions also underway in New York, according to Schaefer and other current and former DEA and Justice Department officials. In interviews, some said their agencies were reluctant to lock horns with the White House as the drawdown loomed. Others cited internal DEA turf battles, and changing agency priorities, including a major refocus on domestic cases. “We just never got our legs back under us to make a go of it again,” said Schaefer, who soon retired too, and joined Schwendiman at SIGAR. Ultimately, the Taliban peace talks went nowhere. Bergdahl was freed a year and four days after Kaidanow’s stand-down order. But in return, Taliban leaders got back the same five senior commanders held at Guantanamo Bay they had demanded at the outset of the talks. “Since the White House ultimately gave the Taliban everything they wanted,” one DEA official asked, “what’s their excuse for not letting us build a case against them all that time?” By the end of 2014, the DEA was down to fewer than a dozen personnel in Afghanistan, rendering obsolete much of a decade’s worth of investigative casework. And as Schwendiman warned, the slashing of DEA support crippled the Kabul government’s ability to counter the narcotics cartels on its own, the DEA officials said. The impact was immediate. In 2015, State Department statistics show, the Taliban — cash rich from record drug profits — surpassed the Islamic State as the deadliest terrorist group worldwide, both in the number of attacks and fatalities. The year before, ISIS was, by far, the leader. Frustrated by the deteriorating situation, Seaman self-published a 104-page manuscript in 2016 detailing 14 years of unfinished DEA counternarcotics investigations in Afghanistan, accusing the Obama administration of placing short-term political points over long-term security interests. He did not go into the circumstances surrounding the shutdown of Operation Reciprocity but was particularly critical of the Obama team’s decision “to seek peace talks with the Taliban in lieu of pursuing them in U.S. courts, when in fact both could have been accomplished simultaneously.” Past administrations, in contrast, had the political will to indict the FARC leadership, he wrote, “exposing their criminality to the world while the Colombian government was involved in peace talks with the FARC” that, years later, ultimately proved successful. None of the DEA and Justice Department officials said their decision to leave Afghanistan, or to retire, was influenced by the abrupt shutdown of Operation Reciprocity. But every one of them told POLITICO that they believe their colleagues called it quits because of their inability to work on the conspiracy case they had spent years building. “We had a lot of successes in spite of the obstacles,” Schaefer said, “but our biggest shot at success was stolen from us.” 

From operations to advice
 Over the past five years, the existence of Operation Reciprocity — and its demise — have remained known only to a relative handful of agents and diplomats. Several current and former U.S. officials said that while they were not familiar with it, the case is emblematic of a far more fundamental conflict at play in Afghanistan — and Iraq — as Washington struggles to contain the hybrid threats that emerged in the post-9/11 world. On one side are U.S. law enforcement officials tasked with investigating drug trafficking, organized crime and corruption at the highest levels in those countries. On the other: Diplomats responsible for seeking accommodation with many of those under suspicion, in order to maintain stability, and a working government. In Afghanistan, many U.S. officials — and numerous coalition allies — supported the DEA’s targeting of Taliban leadership on drug trafficking charges “as a much more strategic and fruitful way forward” than either military force or negotiations, said Candace Rondeaux, a strategic U.S. adviser for SIGAR at the time. But some U.S. diplomats — like Kaidanow — believed spotlighting Taliban leaders’ drug trafficking activities would create too many barriers to an eventual armistice, Rondeaux said. Meanwhile, some Pentagon and CIA officials still protected certain drug lords, citing their purported value as intelligence assets and strategic allies. Such conflicts are to be expected, Rondeaux said. But the Obama administration compounded the problem significantly by not stepping in and protecting its $8 billion-plus counternarcotics investment, despite its 2009 vow to go after “the big guys” atop the drug trade. “This was a problem, fundamentally, of leadership in the White House and a lack of structure within [it] to direct dollars and identify priorities,” said Rondeaux, who is now a senior fellow at the nonpartisan think tank New America’s Center on the Future of War. Behind the story — By Josh Meyer Sometimes one interesting story leads to another. Last year I did a POLITICO exposé about how President Barack Obama’s administration let Hezbollah off the hook. Because of the administration’s zeal for the Iran nuclear deal, criminal investigations into global drug trafficking by Hezbollah’s terrorist wing were derailed. After we published that story, a message popped up in my encrypted email inbox. A former DEA official told me the same thing happened to them but worse… and in Afghanistan. I spent weeks reporting it out, and found that the story was even bigger – and more complicated – than he suggested. This story wouldn’t have been possible without that tip. If you’ve got another tip for me, shoot me a note at my encrypted email account, joshmeyerdc@protonmail.com . One senior Obama-era Pentagon counternarcotics official told POLITICO that high-level administration officials, at the National Security Council and elsewhere, did, in fact, undermine the plan to indict Taliban leaders. “The NSC was aware of it, [and] it ran headlong into the issue of engagement” with the Taliban, he said. Operation Reciprocity, like some Hezbollah cases before it, died from a lack of political support, the official said, rather than an organized conspiracy to shut it down. Even so, the result was the same. “You can kill things by not moving it,” he said. “If DEA is waiting for approval and someone says no, you have to pause, that’s pretty much it.” In interviews, Obama administration officials, including Kaidanow, denied that political tradeoffs undermined Operation Reciprocity or the broader DEA effort in Afghanistan. The White House always favored an “all-tools approach, using law enforcement, diplomacy, sanctions, military, capacity building” in Afghanistan, one senior White House official said. “You can argue that one could have been used more,” the former official said, about the plan for indictments. “But I never got the impression that that [all-tools] approach deviated due to the Taliban discussions” about peace talks. Kaidanow said all decisions on how, and when, to downsize DEA operations in Afghanistan were based on the administration’s overall withdrawal plan, which included transitioning DEA to an advisory role. She also said DEA agents wouldn’t be able to operate throughout the country once military protection and transport were gone. Since the 2014 drawdown, the dozen or so DEA personnel in Afghanistan have, indeed, remained almost entirely “inside the wire,” or within the fortified U.S. Embassy compound, one drug agency official said. “We’re not doing operations there,” just advising Kabul authorities, the official said. Two senior DEA officials acknowledged that shutting down Operation Reciprocity and withdrawing most agents significantly undermined the overall counternarcotics effort in Afghanistan, but said DEA never stopped focusing intensively on top Taliban drug kingpins and their global operations. “I don’t like the fact that the State Department did that, but I knew that we would continue to move forward” by working with government counterparts and its own informants in Afghanistan and other affected countries, one of the senior DEA officials said. “That was all happening as we were in an advisory role,” and led to significant arrests and convictions, including two Afghan traffickers in Thailand in 2015 and a Pakistani drug kingpin in Liberia in 2016. Recently, top Trump administration officials have made several trips to Afghanistan to discuss more troop deployments and peace talks. They included Lisa Curtis, the deputy assistant to the president and senior director for South and Central Asia on the National Security Council. Back in 2013, several months after Operation Reciprocity was halted, Curtis — a former senior Afghanistan hand at the State Department and CIA — wrote a Heritage Foundation policy paper imploring the Obama administration to commit to a long-term counternarcotics effort in Afghanistan as it handed over security operations. “Although the fate of Afghanistan rests with its own people,” she wrote, “without leadership and assistance from the U.S., it will devolve into a narco-terrorist state that poses a threat to regional stability and to the security of the broader international community.” But Curtis, who declined interview requests, and other Trump officials have said little publicly, if anything, about whether the DEA and Justice Department figure into their plans to stop the Taliban and drug lords from seizing control of Afghanistan. ”The U.S. strategy is taking into account the counternarcotics challenge, and the U.S. is considering how to address the issue,” an NSC spokesperson said in response to questions from POLITICO. “Our strategy of confronting the Taliban militarily not only will bring them to the negotiating table but will set the conditions necessary for further counternarcotics effort.” Haidari, the senior Afghan official who helped lead those talks, said his government is wary of a third consecutive U.S. administration promising a military-only solution to its problems. “We felt betrayed,” he said, by Obama officials “who saw no reason to expand counternarcotics, and to go after these kingpins and their accomplices.” The DEA and Justice Department, he said, “should have continued to build upon what they had achieved based on the surge,” especially training programs designed to help Afghans take on the drug networks on their own. Marsac, the former DEA chief, said it would take years to rebuild that capacity in Afghanistan. But a small team of U.S. and Afghan drug agents could make an immediate dent in the insurgency, he said, by using the Operation Reciprocity case files to attack a Taliban-drug nexus that has changed surprisingly little since the project was, essentially, flash-frozen back in 2013. “They asked us to do a job, and we went out and did it,” Marsac said. “It needs to be completed.” *** 

The timeline
 1970s and 1980s Drug Enforcement Administration agents investigate Afghanistan’s narcotics trade but evacuate in 1979 when Soviet troops invade. Opium trafficking skyrockets with help from U.S.-funded Pakistani agents, who deliver weapons to Afghan mujahedeen freedom fighters and help them export their opium. 2002 The DEA leads Operation Containment, a coalition campaign launched after 9/11 to thwart the global narcotics trade by choking off the flow of heroin out of Afghanistan, the world’s leading opium producer, and helping the new Kabul government develop drug enforcement capability. 2005 The DEA takes custody of the first of several Taliban-affiliated Afghan heroin kingpins ultimately tried and convicted in New York courts of overseeing international trafficking organizations importing millions of dollars of narcotics into the U.S. since 1990. Baz Mohammad told co-conspirators that Islamic law approved of their “jihad” to take Americans’ money and kill them through heroin use and addiction. 2007 The DEA helps seize $3.5 billion in narcotics in Afghanistan, up from $1.6 billion in 2005, but the drug trade continues to fuel a massive expansion of the Taliban insurgency and governmentwide corruption. DEA agents double down on tactics they used against Colombia’s FARC narco-terrorists, including military style raids and targeting kingpins with U.S. indictments. 2009 Alarmed by Afghanistan’s inability, or unwillingness, to use its own courts to tackle drug kingpins, Congress funds the biggest-ever international surge of agents in DEA history. More than 80 agents ultimately deploy; three are killed in a November helicopter crash after a major drug raid. 2011 President Barack Obama announces a September 2014 U.S. troop withdrawal and end to the U.S. involvement in the conflict. DEA Kabul soon launches Operation Reciprocity in hopes of quickly decapitating the Taliban leadership before handing over operations to the Afghan government. 2013 DEA Kabul, with support from Justice and State department officials in Afghanistan, unveils a 940-page narcoterrorism prosecution plan to indict 26 Taliban commanders and allied drug lords and try them in U.S. courts. After DOJ’s Terrorism and International Narcotics Unit in New York approves it, a State Department diplomat in Kabul finds out and shuts down all investigative activity in the case. 2016 DEA agents bust a multimillion-dollar Afghanistan-to-U.S. heroin-smuggling ring that informants said had operated for decades. Presidential candidate Donald Trump vows to withdraw from Afghanistan but, once elected, says Taliban leaders and drug kingpins have fostered 20 terrorist groups in the country and threaten U.S. security. 2018 Senior Trump administration officials visit Afghanistan to discuss an additional troop surge and even peace talks with the Taliban but include no plans for incorporating DEA law enforcement efforts as part of their evolving Afghanistan strategy.
"""

# Text 2. | https://www.politico.eu/article/fidel-castro-lisa-howard-my-dearest-fidel-a-journalists-secret-liaison/
text2 = """Lisa Howard had been waiting for more than two hours in a suite of the Hotel Riviera, enough time to bathe, dress and apply makeup, then take it all off to get ready for bed when  she thought he wasn’t coming. But at 11:30 p.m. on that night in Havana — February 2, 1964 — Howard, an American correspondent with ABC News, finally heard a knock at the door. She opened it and saw the man she had been waiting for: Fidel Castro, the 37-year-old leader of the Cuban revolution and one of America’s leading Cold War antagonists. “You may be the prime minister, but I’m a very important journalist. How dare you keep me waiting,” Howard declared with mock anger. She then invited Castro, accompanied by his top aide, René Vallejo, into her room. Over the next few hours, they talked about everything from Marxist theory to the treatment of Cuba’s political prisoners. They reminisced about President John F. Kennedy, who had been assassinated just a few months earlier. Castro told Howard about his trip to Russia the previous spring, and the “personal attention” he had received from the “brilliant” Soviet Premier Nikita Khrushchev. Howard admonished Castro for the repressive regime he was creating in Cuba. “To make an honorable revolution … you must give up the notion of wanting to be prime minister for as long as you live.” “Lisa,” Castro asked, “you really think I run a police state?” “Yes,” she answered. “I do.” In the early morning hours, Howard asked Vallejo to leave. Finally alone with her, Castro slipped his arms around the American journalist, and the two lay on the bed, where, as Howard recalled in her diary, Castro “kissed and caressed me … expertly with restrained passion.” “He talked on about wanting to have me,” Howard wrote, but “would not undress or go all the way.” “We like each other very much,” Castro told her, admitting he was having trouble finding the words to express his reluctance. “You have done much for us, you have written a lot, spoken a lot about us. But if we go to bed then it will be complicated and our relationship will be destroyed.” He told her he would see her again — “and that it would come naturally.” Just before the sun rose over Havana, Castro tucked Howard in, turned out the lights and left. Howard’s trip to Havana in the winter of 1964 was pivotal in advancing one of the most unusual and consequential partnerships in the history of U.S.-Cuban relations. She became Castro’s leading American confidant, as well as his covert interlocutor with the White House — the key link in a top-secret back channel she singlehandedly established between Washington and Havana to explore the possibility of rapprochement in the aftermath of the Cuban missile crisis. From mid-1963 to the end of 1964, Howard secretly relayed messages from Cuba’s revolutionary regime to the White House and back again; she also used her reporting skills and high-profile perch at ABC to publicly challenge the Cold War mindset that Castro was an implacable foe of U.S. interests. Her role as peacemaker was built on a complex, little-understood personal rapport she managed to forge with Castro himself — a relationship that was political and personal, intellectual and intimate. Today, almost no one remembers Lisa Howard. But in the early 1960s, she was one of the most famous female TV journalists in the United States — a glamorous former soap opera star who reinvented herself as a reporter and then climbed to the top of the male-monopolized world of television news. She became ABC’s first female correspondent and the first woman to anchor her own network news show.  Her influential role in the media empowered her efforts on Cuba, even as it worried White House officials who were the targets of her ceaseless pressure to change U.S. policy. In top-secret reports from the era, those officials speculated about “a physical relationship between” Howard and Castro and feared she would use her position at ABC News to break the story of Washington’s secret talks with the Cuban comandante. But both she and Castro took the secret of their intimate diplomacy to their graves. Only now, thanks to declassified official documents and, most important, Howard’s own unpublished diaries and letters, can the story finally be told of how one tenacious  journalist earned the trust of the legendary leader of the Cuban revolution, and cajoled two U.S presidents into considering peaceful coexistence with him. *** Lisa Howard was born Dorothy Jean Guggenheim to a middle-class Jewish family in Ohio, but she was first known to the world as TV’s “first lady of sin” — a designation Hollywood bestowed on her for playing temptresses, murderesses and thieves in forgettable TV programs and second-rate movies in the early 1950s. In 1957, she scored the recurring role of Louise Grimsley in the popular CBS series “The Edge of Night.” But even as she gained attention in Hollywood, Howard signaled far greater ambitions. “Though a looker (5’3; 109 lbs; 35-23-35 from bust to hips),” read a cringeworthy 1953  cover story in People Today, “Miss Sin prefers to think of herself as the ‘sensitive-intellectual type’ who is ‘going places.’” And she was. “I became more and more interested in politics and world affairs … and less and less interested about the fate of Louise Grimsley,” Howard would later recount to audiences on the lecture circuit. “I wanted to talk to people who were making news. I wanted to be there on the spot when history was being written.” So, in 1960, while living in New York City with her husband, Walter Lowendahl, and two daughters, Howard abandoned her acting career, grabbed a tape recorder and began scoring exclusive radio interviews as an unpaid volunteer for the Mutual Radio Network. She earned access to major political figures, including then-Senator John F. Kennedy, former first lady  Eleanor Roosevelt  and even President Dwight Eisenhower. But it was Howard’s lengthy interview with Khrushchev in September 1960 — the first the Soviet leader had granted to a reporter from the West — that caught the attention of executives at ABC News. In May 1961, ABC hired Howard, then 35, as its first-ever female correspondent; two years later, the network gave her her own show — a daily  midafternoon broadcast geared toward housewives called “Lisa Howard and News with the Woman’s Touch.” At a time when women in television news were typically relegated to reporting on fashion, lifestyle and the weather, Howard’s was the first female face beamed into the living rooms of America offering authoritative coverage of national and international events on a daily basis. “Six changes of Puccis and six politicians in one day are par for the course for Lisa Howard,” read a 1963 McCall’s Magazine  cover story about Howard that described her as “a dead-serious reporter,” as well as “bright, buxom, and bumptious.” In another profile that same year, Time magazine wrote that the pioneering female journalist “has achieved this distinction by scrambling harder than six monkeys peeling the same banana. … Political leaders, domestic and foreign, have learned that there is no dodging Lisa Howard.” In the early 1960s, the Cuban leader was one of the most dynamic, and for U.S. policymakers, alarming, new figures on the international political stage. The young, bearded guerrilla fighter had overthrown the U.S.-backed authoritarian regime of Cuban President Fulgencio Batista on January 1, 1959, installing a revolutionary government  just 90 miles off the coast of Florida. Initially, the United States seemed impressed with Castro’s charisma. But U.S. officials soured on his anti-American rhetoric and his economic outreach to the Soviet Union. In the spring of 1960, Eisenhower authorized planning for a secret CIA paramilitary intervention to roll back the Cuban revolution and install a more compliant government in Havana,  cutting diplomatic relations in January 1961. Kennedy inherited the covert operation, gave it the green light to proceed in April 1961 at the Bay of Pigs, and watched it explode into a major debacle when Castro’s militia defeated the CIA-led brigade in less than 72 hours. In frustration, he ordered a new program of covert operations against Cuba, known as Operation Mongoose, and a full economic blockade in early 1962 — aggressive moves that persuaded Castro, who had recently declared Cuba a socialist state, to accept Soviet nuclear missiles as a deterrent to another U.S. invasion, leading to the Cuban missile crisis. For 13 days in October, the world stood on the brink of nuclear Armageddon until Kennedy offered Khrushchev a secret deal: pulling U.S. missiles out of Turkey in exchange for removal of the missiles in Cuba. With Castro furious at Khrushchev for removing the weapons without consulting him, some Kennedy officials saw the opportunity to entice Castro back into the Western orbit; the CIA, however, was determined to continue efforts to overthrow him. Cuba was a major news story. But with tensions running high, the embargo in place and no direct travel between the two countries, few establishment reporters could gain access to the country, let alone an interview with its fiery leader.  Howard had tried and failed to obtain an interview with Castro twice in the early 1960s, and after the missile crisis she made another attempt. “Considering the present state of the world crisis,” she wrote Castro, “wouldn’t this be an ideal moment for you to speak to the American people?” After months of her cajoling, the Cuban mission in New York finally granted Howard a visa to travel to Havana in early April 1963. Castro ignored her for several weeks as he finished negotiations with New York lawyer James Donovan for the release of U.S. prisoners in Cuban jails and prepared to take a long trip to Russia for his first summit with Khrushchev. In a bid for his attention, Howard wrote Castro a letter after she arrived — “I beg you to say ‘YES,’” it stated in Spanish. “Give me this interview, please” — and passed it on to various interlocutors, among them Donovan, whom she beseeched to put in a good word for her. “I told [Castro] there was a beautiful blonde dish of a reporter wanting to interview him and would he give her some of his time,” Donovan recalled. “I went about it by whetting Castro’s natural masculine curiosity and vanity.” Whether out of curiosity and vanity, or a sense that Howard could become a genuinely valuable channel to America, Castro relented and agreed to meet Howard at the nightclub in the Havana Riviera hotel. He arrived at midnight on April 21, and the two talked until almost 6 a.m., discussing Kennedy, Howard’s personal impressions of Khrushchev — “a sly old fox” who “would cut you off like a twig” — and what Howard deplored as “the police state apparatus” under Castro’s rule. Howard was impressed by Castro’s breadth of knowledge. “Never, never have I found a Communist interested in the sentiments of Albert Camus,” Howard later recounted in a letter. “And I certainly have not found dedicated Communists anxious to discuss the merits of our Constitution and our Bill of Rights. But Fidel enjoyed the conversation immensely.” Castro enjoyed the conversation so much that he agreed to a formal interview — the first he had granted a U.S. television journalist since 1959. In the early hours of April 24, with  Cuban Communist Party cameras rolling at the Riviera, Howard put a series of forceful questions to the Cuban comandante: When had he become a communist? Did he ask Khrushchev for the nuclear missiles? Why were hundreds of thousands of Cubans fleeing to Florida? There were lighter moments, too. Castro asked Howard whether her bright blond hair color was natural. “We don’t have to answer questions like that in my country,” she shot back.  And then came the showstopper: Under what conditions might he support a rapprochement with Washington? Castro cited his successful talks with Donovan on the prisoner release as a positive step forward. A rapprochement “was possible,” he noted in halting English, “if the United States government wishes it.” Coming from one of America’s most renowned Cold War enemies, just months after a tense nuclear standoff, Castro’s interest in better relations was headline news. Within hours of the interview, Castro flew off to Moscow — but not before he had arranged for a huge bouquet of flowers to be delivered to Howard’s hotel room. In return, the journalist left Castro what she described as “a little keepsake” — a deeply personal letter she drafted in her room at the Riviera. “I wanted to give you something to express my gratitude for the time you granted me; for the interview; for the beautiful flowers,” her message began. “I have decided to give you the most valuable possession I have to offer. Namely: my faith in your honor. My faith in the form of a letter, which, if revealed, could destroy me in the United States.” Howard described her four-page letter, the drafts of which she saved along with other records from her trip, as “a tribute, a poem to you — the man.” It mixed intense criticism with sincere praise. “I do not want you destroyed. … You possess what George Bernard Shaw called ‘that spark of divine fire,’” Howard wrote. “You are not the ruthless, cynical tyrant [your critics] have depicted. … I do not believe you have meant to hurt people, though, in all candor, I am both saddened and outraged that you have destroyed thousands and harmed many more without just cause.” Howard beseeched Castro to find his “way back” — to become the transformational historical figure she believed to be his destiny. “What you have to offer the world that is meaningful and universally applicable is not some capricious brand of tropical Marxism (the world scarcely needs that), but your humanity; your compassion; your deep knowledge and sense of justice; your genuine concern for the poor; the sick; the oppressed; the defenseless; the lost; the despairing. … And your sacred duty, your solemn obligation to mankind is to make that quality ever stronger, to make it a reality for your people — all your people, every class and sector. Let flow in the most untrammeled way the goodness that is your substance and can be your salvation.” “I feel deeply that you must be permitted to play out your role,” Howard continued, pledging to do what she could to ensure Castro’s survival and bring the U.S. and Cuba together. “I am going to talk to certain people when I return to the States,” she wrote. “I do not overestimate my influence. But I shall try to help.” One draft of her message, typed on Hotel Riviera letterhead, ended “on a personal note.” “We met and came together and, I know, felt something for one another that could not go further. I am who I am and you are Fidel Castro and for us, at this moment in history, nothing personal could be realized. No matter … our personal desires are not important.” Howard crossed out that paragraph during a revision, big blue Xs cutting through the type. “Perhaps we shall never see one another again,” the letter concluded instead. “But I shall treasure with all my heart for as long as I live my trip to Cuba in April of 1963 and my meetings with you, my dearest Fidel.” *** When “Fidel Castro: Self Portrait” aired on ABC on May 10, 1963, it dominated the news cycle. “Castro applauds U.S. ‘Peace Steps,’” declared the New York Times. “Castro Would Like Talk With Kennedy,” announced the Cleveland Plain Dealer. “The interview was a great success, front page of nearly every paper in the country,” Howard wrote in a private note for Castro. “The entire interview is now being discussed on the highest levels.” That was only the public part of the message she delivered. Behind the scenes, as she had promised, Howard met with CIA and State Department officials to personally convey Castro’s interest in a dialogue with the United States. She used the positive news coverage of her ABC interview to argue that public opinion was not opposed to better relations with Cuba, and even presented a list of potential intermediaries who could facilitate talks with Castro — including herself. “Liza [sic] Howard definitely wants to impress the U.S. Government with two facts: Castro is ready to discuss rapprochement and she herself is ready to discuss it with him if asked to do so by the U.S. Government,” stated a secret CIA report delivered to the White House. Howard also typed out a 10-page brief to Kennedy himself, elaborating on what Castro had told her during their conversations in Havana  and attempting to obtain a meeting. “I wanted to see you personally,” she wrote, “to impress upon you how strongly I feel that Fidel’s alliance with the Communists is a precarious one … [and] that we might profitably fish in those troubled waters.”  Castro was “now ready to discuss everything: the withdrawal of [Soviet] troops; an end to the exporting of his revolution” to end the blockade and resume diplomatic relations with the United States, she reported. “And not just ready, Sir, but positively eager.” “He was most interested in you, Mr. President,” Howard continued. “He kept saying to me ‘What is President Kennedy like, what does he want … what does he want of us?’” She beseeched Kennedy to actually “sit down and negotiate with Fidel.” Unbeknownst to Howard, however, the CIA vigorously opposed her message of potential reconciliation — and lobbied Kennedy to ignore it. In a secret memo to the White House, dated May 2, CIA Director John McCone recommended that “the Lisa Howard report be handled in the most limited and sensitive manner” and “that no active steps be taken on the rapprochement matter at this time.” Howard’s initial efforts went nowhere. But she would not be ignored, nor denied. As McCall’s wrote about Howard that year, “Her massive drive is so uncomplicated and single-minded that it usually carries the day.”  “The key to understanding Lisa,” her husband told the magazine, “is to think of her as a sort of mutation. She simply doesn’t have the inhibitions other people have. When she goes after something, she’s completely overt, she has no qualms or second thoughts or reluctance to operate.” Getting no traction at the White House, Howard redrafted her letter to the president into an article, “Castro’s Overture,” which appeared as a cover story in the September 1963 issue of the liberal journal War/Peace Report. Castro had been “emphatic about his desire for negotiations” in their conversations, she reported. She called on Kennedy to “send an American government official on a quiet mission to Havana to see what Castro has to say.” At the United Nations, a U.S. official named William Attwood read Howard’s article. As a former senior editor of Look magazine, Attwood had interviewed Castro in 1959 and shared Howard’s view that coexistence with the Cuban regime was both possible and preferable. On September 12, he called her, and together they set in motion a plan of action. First, Attwood approached U.N. Ambassador Adlai Stevenson to get a green light from Kennedy to make “discreet contact” with Cuba’s ambassador to the United Nations, Carlos Lechuga. Then, Howard approached Lechuga in the U.N. lounge and told him that Attwood urgently wanted to talk to him. A cocktail party at her East 74th Street town house would serve as cover for the two diplomats to meet. On September 23, as members of the New York literati munched on finger foods and sipped drinks in Howard’s house, the United States and Cuba held their first, albeit informal, bilateral meeting since the Eisenhower administration. Off in a corner of the living room, Attwood and Lechuga discussed how negotiations between their two hostile countries might be initiated. Lechuga “hinted that Castro was indeed in the mood to talk,” Attwood reported back to the White House, adding that “there was a good chance I might be invited to Cuba.” Over the next two months, Howard’s home became the hub for secret communications between the U.S. and Cuba.  Howard placed a series of calls to Castro’s office conveying U.S. interest in setting up a meeting, and passed on Castro’s responses to Attwood. Finally, Howard set up a time for Attwood to talk directly with Castro’s top aide, Vallejo. When Attwood arrived at midnight on the evening of November 18 for the call, Howard greeted him wearing a lavish dressing gown. As she dialed Cuba again and again, attempting to track down Vallejo, they listened to jazz, drank bourbon and discussed French philosophers. In her diary, Howard recorded this dramatic turning point in her protracted efforts to connect Washington and Havana: “‘D’ Day for the telephone appointment. … We put through the call. No Vallejo. Placed at least seven calls. … Read Camus out loud. … Me on the bed in a lacey peignoir — Bill sipping bourbon and shy but dying to slip into bed with me. And there was that white phone — mute — tense … our link to our secret and oh-so-longed for mission. We do have a deep common bond. An inexorable conviction that this can be an honorable rapprochement between Cuba and the U.S.” Around 3 a.m., Howard managed to reach Vallejo and put Attwood on the line to discuss arrangements for the two to meet clandestinely. This was the moment Howard had long awaited. “At last! At last! That first halting step. Contact has been established!” she rejoiced in her diary. “I feel strongly this is only the beginning. A long, frustrating, tension-filled, but exciting experience lies ahead.” *** Three days later, Howard found herself covering the shocking story of Kennedy’s assassination for ABC. Howard, Castro and a handful of U.S. officials knew the assassin’s bullet had terminated not only JFK’s life, but also his secret efforts to find common ground with Cuba. “The events of November 22 would appear to make an accommodation with Castro an even more doubtful issue than it was,” wrote National Security Council aide Gordon Chase in a Top Secret/Eyes Only White House assessment. “In addition, the fact that Lee Oswald has been heralded as a pro-Castro type may make rapprochement with Cuba more difficult.” But Howard would not give up. She persuaded her superiors at ABC to let her return to Cuba to do another TV special — this time on life under the revolution. When she informed the new administration about her trip, White House staff responded that they would be interested in what Castro had to say. Howard and her entourage arrived at José Martí International Airport on February 1, 1964. Castro had sent Vallejo to meet her, and “I was taken through customs like a diplomat,” she recalled. She was a diplomat — albeit a self-appointed one. While filming the new TV special, she had also be strategizing with Castro about how to renew his delicate diplomacy with President Lyndon B. Johnson. There was another reason she was eager to be in Havana. “When will I see him?” Howard asked Vallejo upon her arrival. “He has been crazy to know when you’re arriving,” the aide replied. “He’s been asking about you all day.” She didn’t see Castro until the next evening, February 2, 1964, when he arrived at her hotel close to midnight and the two stayed up until dawn before he tucked her in and left. Over the next two weeks, Howard and her crew traipsed around Cuba with the energetic Castro, filming him playing baseball, visiting a cattle farm and interacting with peasants. As much as Howard believed Castro was a dictator, the overwhelming public adoration he generated impressed her. “They mob him, they scream ‘Fidel, Fidel,’ children kiss him, mothers touch him,” she wrote. “They are awed, thrilled … ecstatic, but mostly passionate. There is no doubt in my mind that the emotion Fidel inspires in all women is sheer undiluted sexual desire. He is the most physical animal man I have ever known.” The attraction between them was undeniable. “I sat and stood beside him for five hours and I nearly went out of my mind,” she recounted. One night, Howard returned to her suite and burst into tears, torn between her feelings for the man and her distaste for his revolution. “This revolution isn’t at all what he thinks it is,” she wrote in her diary. “How can I tell this to Fidel. And why do I feel that I must? Yet I guess what keeps me involved is that down deep I believe that if I could convince him of the truth … of the despair and agony and chaos he has brought to this Island … he would change.” Certainly, she believed, ending the existential threat that Washington’s hostility posed to Castro would contribute to that goal. During their formal ABC interview in the wee hours of February 13, Howard posed a question to which she already knew the answer: “You said at one point after President Kennedy’s death that you believed that under Kennedy it was going to be possible to normalize relations between Cuba and the United States. What leads you to believe that?” “My opinion is that he was in the way of persuading himself of his mistakes about Cuba,” Castro responded in stilted English. “We had some evidence that some change was taking place in the mind of the government of the United States ... I do not want to speak about now.” It was well after midnight when the interview finished, and Castro, Howard and Vallejo adjourned to the bedroom of Howard’s suite. “We were in a wonderful mood,” Howard wrote in her diary. The Cuban leader lay down on the sofa and put his head in her lap. “[Secretary of State] Dean Rusk should see us now,” Howard joked, as Castro roared with laughter. Lounging on the couch, they strategized about how to entice Johnson to finish the dialogue Kennedy had started. Castro said he wanted to “discuss a trade” with the new administration: The United States would stop backing sabotage raids into Cuba led by Cuban exiles in Florida  and halt its effort to roll back the Cuban revolution. In return, Cuba would end its efforts to export revolution to other areas of Latin America. Castro also said he would do what he could to ensure Johnson was elected in November 1964,  rather than face the prospect of a hard-line Republican such as Senator Barry Goldwater as president. If the Johnson administration “feels they must take some hostile action for domestic political consumption,” Castro said he would even understand. “If he was informed, ex-officio, that this was a political action,” he would refrain from retaliating. At 3:30 in the morning, Howard once again decided it was time for Vallejo to give them some privacy, which made Castro nervous. “I can’t be alone with you without my lawyer,” he joked. When Howard announced she wanted to “get into something comfortable,” he made a futile attempt to keep her fully clothed. “He made a great fuss about my not changing my dress because it was so pretty and he wanted to look at it,” she wrote. And when she emerged from the bathroom in a nightgown and pajamas, he chastised her for disobeying him. “You don’t understand me,” he complained in a flourish of machismo. “You just want to do what you want to do. Why can’t you treat me like a man?” Castro turned the conversation to their complicated relationship. Nights earlier, Castro had confided that he used to sleep with many women, but not anymore — “that now that he is the leader all the women want to go to bed with him, but he thought it wasn’t him they wanted but to sleep with the leader. This seemed to trouble him,” Howard recounted. As Castro explained why he was reluctant to sleep with her, he asked Howard: “What do you want, Lisa? Do you want my body?” Tonight, he was still conflicted. “He said he wanted me very much but the conditions had to be right and we had to be away somewhere where we could forget everything,” Howard wrote. Nevertheless, “we did get to bed and he made love to me quite expertly and it was, of course, thrilling and ecstatic — as much as anything I have ever experienced.” “Lisa, you are not simple,” Castro told Howard just before he left. “With you and me it is not simple. But that is more interesting.” They would engage in one more conversation on that trip, an emotional tête-à-tête as Howard readied to leave two mornings later. Castro arrived at Howard’s hotel suite at 5:30 a.m. to ensure she made her flight, and found her drugged from a sleeping pill, unable to wake up. While she, half asleep, entreated Castro to delay her flight — a request he refused, telling her “that would be arbitrary” — he managed to rouse her. “I dressed in front of Fidel like he was a schoolroom mate,” Howard recalled. Then, “he pulled me over and asked me to sit on his lap, and then spoke to me very gently, and said, ‘Lisa, you are very dangerous for me. I could love a girl like you very deeply. You’re very sweet, very pretty, very intelligent, very sensitive.” If they were together, he suggested, “we would have many fights, a hundred fights, two hundred fights, but in the end it would be all right.” He said: “You can teach me very much.” Howard told Castro he had “touched [her] very deeply.” But she confessed to being “overwhelmed by sadness” watching him intermingle with Cuban citizens because “he had such a genuine belief in the revolution and in what he was doing [when] in fact so much of what he was doing was truly evil.” He could not see it, “and I was not capable of making him see it,” she tearfully explained. “Castro said he understood part of what I was trying to say, and that I must return again and we must talk and talk and talk for many, many hours and days,” she wrote. He promised to take English lessons so that they could “understand each other better.” *** Once back in New York , Howard typed up a six-paragraph memo to Johnson from Castro, titled “Verbal Message given to Miss Lisa Howard of ABC News on February 12, 1964 in Havana, Cuba.” In the missive, Howard relayed what she and Castro had discussed in her suite — from Castro’s offer to weather a U.S. provocation during the campaign to his hope to continue the dialogue Kennedy had started. He recognized the need for “absolute secrecy,” and suggested that Howard could be trusted as an intermediary. With the memorandum in hand, Howard placed a call to Gordon Chase at the NSC, now her contact in the new administration, and told him she had a confidential message for Johnson. “Lisa Howard wants very much to give her message from Fidel to the President only,” Chase reported to the national security adviser, McGeorge Bundy. Bundy, however, was dubious of Howard’s dual role as a secret go-between and a prominent journalist. “She is an extraordinarily determined and self-important creature and will undoubtedly knock at every door we have at least five times,” he warned other White House officials. “It is quite impossible that she can see Castro and the president without writing about her peacemaking efforts at some stage, and I see nothing whatever to be gained by letting her play this game with us.” Chase, however, pressed Bundy for permission to debrief Howard and try to “pump out” Castro’s message. As “a shrewd, aggressive, good-looking gal,” he argued, “she probably gets a lot closer to Fidel than most (pure speculation) and may be able to give us some insights about Castro’s intentions.” On March 7, Chase traveled to New York to receive a briefing on Howard’s trip. As they pored over photographs and the transcripts of her interviews, they agreed on a common mission “to get Fidel to end his Soviet tie and end exporting the Rev[olution] and announce elections in exchange for a guarantee of American aid, trade, and official recognition.” Howard offered her services as an “effective emissary” and affirmed her discretion. “So the young man will make his report to Bundy and we shall see,” she wrote. In his comprehensive Top Secret/Eyes Only report on their meeting — titled “Mrs. Lisa Howard” — Chase advanced Howard’s message “that we should be communicating with Castro” about normalized relations. “I regard Mrs. Howard’s motives as mixed,” he advised: “First, she is a newspaper woman and probably knows she is sniffing at a highly readable story. Second, because of her influence with Fidel, she probably regards herself, somewhat romantically, as fated to play a historical role in helping to bring about an agreement between the U.S. and Cuba. Third, she probably is a sincere, anti-communist, libertarian democrat who regards the Cuban scene as a tragedy and who wants to see the island living in the Western tradition and at peace with the U.S. (To go out on a dangerous limb, my own estimate is that as long as she can feel useful, the last two motives control the first.)” Chase transmitted Howard’s assertion that she had “a rapport with Castro which a man will not easily duplicate. I am not certain that there is a physical relationship between them,” he informed Bundy, “but regard it as likely.” Sensing she now had a strong ally inside the White House, Howard began placing evening calls to Chase at his home, seeking his help to obtain a meeting with Johnson so that she might deliver Castro’s message. Each time, Chase gently put her off and tried to persuade her to entrust the message to him, which she declined to do. In a top-secret memo on these conversations, Chase reported, “She roundly scolded me and the White House for taking her message from Fidel to the President as a joke. I assured her we didn’t.” Stymied at the White House, Howard turned again to U.N. Ambassador Stevenson. Late in the evening on June 5, 1964, she went to see Stevenson at his room in the Waldorf Astoria. The two discussed how to persuade Johnson to continue dialogue with Cuba. She gave him Castro’s “verbal message” and entrusted him to personally transmit it to the president. True to his word, on June 16, Stevenson sent LBJ a top-secret memorandum, with Castro’s secret communique — one of the most compelling Castro ever sent to a U.S. president — attached. Stevenson advised the president of the secret dialogue Kennedy and Castro were pursuing at the time of the assassination and recommended that “if it could be resumed on a low enough level to avoid any possible embarrassment, it might be worth considering.” *** Three days later , Howard traveled to Cuba for the third time — this time not as an ABC journalist but as a secret emissary. Her mission was to report to Castro that she had finally gotten his message into Johnson’s hands. But she also carried a high-level warning from the White House: The U.S. government was concerned about threats Castro had made to shoot down U.S. reconnaissance planes that continued to overfly Cuba in the wake of the missile crisis. Castro arranged for Howard to stay in one of the confiscated mansions that now served as a protocol house. The house came with a Cadillac and chauffeur, a butler and cook, air-conditioned bedrooms and a sunken bathtub. She had come a “long way from ‘Edge of Night’ to guest of the Cuban government,” Howard confided in her diary. Far less luxurious was their one evening spent together on Castro’s “yacht” in the Bay of Pigs, which Howard described as a small, battered boat with a broken shower that slept two. They stayed up till 5:30 a.m. talking about “politics, life, love, freedom, peace, hope, despair, my family, all of it,” Howard remembered. They also discussed the U.S. warning to refrain from shooting at any U.S. reconnaissance planes. Castro promised to restrain himself during the 1964 election season. “You are right, Fidel,” she later confided to him about their night on the boat. “Our intellectual relationship is the essential one. Though the other one is rather pleasant too … the frosting on the cake.” Before she left Havana, they talked over how their back channel would work: To prevent future incidents between the United States in Cuba, Castro would rely on Howard to get messages to Stevenson and would count on his response, passed through her. Less than two days after she returned to the states, Castro used this channel to address a crisis at Guantanamo, where a U.S. Marine had reportedly shot a Cuban soldier. On June 26, Vallejo placed an urgent call to Howard and shared his leader’s message: “Please call Governor Stevenson and tell him about the shooting, that the Cuban is in the hospital and Castro thinks he is going to die, that this is the second time there has been a shooting at the base. He wonders if it is part of a deliberate plan of provocation or an isolated act.” Howard immediately called Stevenson. She told him what Castro had said about reconnaissance planes and asked for an answer on the shooting. He assured her “there was no plan whatsoever of deliberate provocation at the Guantanamo Base.” She then relayed the report to Vallejo. “Fidel was glad to get my message,” Howard wrote in her diary the next day. “I guess he feels our channel of communications has been established.” Indeed, the back channel — known as the “Castro / Lisa Howard / Stevenson / President line” in top-secret White House documents — between the White House and the Cuban leadership was now open, and active. In a top-secret memo to Johnson written after the phone call, Stevenson reported Castro’s message that “there will be no crisis until after the November elections; that nothing will happen to our [reconnaissance] planes, and that we do not need to send him any warnings. He will use utmost restraint and we can relax.” Stevenson also conveyed Castro’s belief that “all of our crises could be avoided if there was some way to communicate; that for want of anything better, [Castro] assumed that he could call [Howard] and she call me and I would advise you.” *** Howard had almost single-handedly built an unprecedented bridge between Castro and the Oval Office. But the White House wasted no time shutting her out. In a July 7, 1964, memo to Bundy, Chase warned that the newly established communications “make Lisa Howard’s participation even scarier than it was before. … Before this, the Johnson Administration had relatively little to fear from Lisa since, essentially, we were just listening to her reports on or from Castro.” Chase also warned Stevenson’s involvement would mean more media attention if news of the back channel leaked. “Lisa’s contact on the U.S. side is far sexier now (Stevenson), than at any time in the past (Attwood and then Chase).” Extricating Howard from these secret operations without offending her and risking public exposure of U.S.-Cuba communications, Chase understood, would be a delicate operation. “Lisa should relax, stay quiet, and stand at the ready,” was the message Chase recommended passing to her. “We may want to use her influence with Castro in the future.” It is unknown whether that message was ever delivered, but after July 1964, the Johnson administration appears to have cut Howard out of the loop. There are no more memos about contacts between Howard and Castro — and no more diary entries about communications with the White House. As for official communications with Cuba, U.S. officials turned a deaf ear to Castro’s public call for “extensive discussions” with Washington, and to an offer from his brother, Raúl Castro, to meet with U.S. negotiators “any place to discuss improving relations, even the moon.” Deeply frustrated, in December 1964, Howard seized on the visit of Che Guevara, the Argentine revolutionary who had helped usher in the Cuban revolution, to the United Nations to renew her attempts to bridge the Cold War gap across the Florida straits. She shepherded Guevara around town — together they attended a premiere of a new documentary film commemorating the life of Kennedy — and organized a soiree for him at her New York apartment. “Che Guevara has something to say” to the White House, she told Chase on the phone, in hopes of once again using cocktail diplomacy as a cover for the two sides to confer. “I asked her point blank whether this was her idea or Che’s,” Chase reported to his superiors. “She would not answer me directly and kept repeating that she was ‘in a position to arrange a meeting.’” “Stevenson was all hot to go on this,” according to a top-secret White House memo, after Howard invited the U.N. ambassador to talk with Guevara. But State Department officials refused to authorize a Stevenson-Guevara meeting for fear it would quickly leak to the press. Howard did manage to persuade the progressive senator from Minnesota, Eugene McCarthy, to come to her cocktail party and talk to Guevara off in a corner. “The purpose of the meeting was to express Cuban interest in trade with the U.S. and U.S. recognition of the Cuban regime,” McCarthy reported to the State Department the next day. But after debriefing him, U.S. officials concluded that “the conversation was entirely Lisa-generated and that Che really had nothing to tell us.” *** As Howard lost her Cuba cachet with the Johnson administration, she also lost her position at ABC News. In late September, as the 1964 election approached, the network summarily suspended her, citing her public participation in “Democrats for Keating” — a committee of prominent New York liberals who opposed Robert Kennedy’s bid to become a senator from their state. After the election, ABC fired her. When Howard moved to sue ABC for violating her constitutional rights to express her political beliefs, ABC executives let it be known that “her actions regarding the Cuba show” were among their reasons for terminating her contract. Indeed, Howard’s internal struggle to control the tone and content of her April 1964 TV special, “Cuba and Castro Today,” marked the beginning of her downfall at ABC. According to Howard, she had waged “a titanic battle” with network executives to keep the broadcast from adopting a conventional Cold War approach to the complex issue of the Cuban revolution. “We fought over every inch of the show,” she recorded in her diary. ABC higher-ups — in particular the executive director of news, Jesse Zousmer — wanted “to present just one more indictment of Fidel Castro and his revolution,” she wrote. “I could not do that. I would not do that.” When the program finally aired, Howard believed she had “won all the major points.” The broadcast was “not an indictment of Fidel — and he comes off fairly well,” she wrote. Most important, “I think it will help U.S.-Cuban relations.” Howard might have won the battle over her TV special, but in the ensuing weeks and months, she lost the war. Within the news division, Zousmer became a powerful, and in Howard’s mind, “brutally vindictive” foe. In mid-April, as the special was being finalized, Howard twice failed to appear for her daily show, and Zousmer circulated a memo stating that he planned “to take definitive action” if she failed to honor her contractual obligations. “You have tried to bully me, insult me and humiliate me,” Howard responded  in a blunt memo to her boss. “I strongly advise you not to threaten me again.” According to Howard, Zousmer began chipping away at her job. During the July 1964 Republican National Convention in San Francisco, she received few assignments, and the interviews she did were not used on the evening news. By the time of the Democratic National Convention in late August, Howard had initiated Democrats for Keating and was lobbying party leaders not to support RFK’s Senate bid. During the convention, ABC received two calls from the White House press secretary, Pierre Salinger, complaining that Howard was creating “quite a stir” by speaking out against Kennedy. ABC dispatched an executive to the convention in Atlantic City, New Jersey, to tell her to cease and desist. Never one to compromise her principles, Howard escalated her public efforts against Kennedy’s candidacy. “I can assure you that I am acting in my capacity as a United States citizen and my television broadcasts will in no way reflect my personal involvement,” she wrote her superiors in defense of her political activities on September 16, 1964. Without warning, two weeks later ABC suspended Howard from her daily show. Less than a month later, she was fired. Her efforts to get ABC to reconsider failed, as did her attempts to get a job at another network. One ABC executive informed her “she had been marked as ‘lousy.’” A civil suit Howard filed against ABC seeking $2 million in damages to her reputation and career was dismissed by the New York Supreme Court in early 1965. And then came the personal tragedy. In the late spring of 1965, Howard suffered a miscarriage. Her ensuing depression resulted in a period of hospitalization that, sadly, failed to relieve her despondence. On July 4, 1965, while spending the holiday weekend in the Hamptons, Howard altered a prescription for 10 barbiturates and obtained a bottle of 100 tablets at a local pharmacy; she consumed the pills in the parking lot and died of the overdose. She was 39 years old. The FBI would soon launch a bizarre inquiry to determine whether her death was somehow tied to Guevara’s disappearance following his visit to New York. (Unbeknownst to the U.S. intelligence community, Guevara had gone underground to lead guerrilla fighters in the Congo.) FBI agents interviewed Howard’s former colleagues at ABC about her Cuba work, her relations with Castro and Guevara, and why she might take her own life. The FBI also reviewed her case with members of the NYPD to ascertain whether Howard’s was “a legitimate suicide” — or sinister foul play tied, presumably, to her work on Cuba. Howard's Cuba work is a fundamental part of her forgotten legacy. “I was an integral part of this fledgling new look at Cuba,” Howard once confided. Her efforts might not have fully paid off during her short lifetime, but they created the historical foundation for the back-channel diplomacy that led to the breakthrough in relations achieved by the Obama administration 50 years later. As Cuban President Raúl Castro steps down from power in April, and as U.S. policy makers revisit relations with Cuba in a post-Castro era, Howard must be remembered as an essential player in the original efforts to bring about what, in her diary, she called “an honorable rapprochement.” “She showed us by her extraordinary sacrifice what moral strength means,” Senator William Proxmire said in his eulogy at Howard’s memorial service, without even knowing the extraordinary role she had played behind the scenes. “To live by the truth as she saw it; to dig out more of what she regarded as the truth than the establishment can comfortably permit. And to speak that truth loud and clear.” Castro recognized her fearlessness, too—and knew what it had been able to accomplish. “You know no one could come down here and do what you did — with your will and persuasiveness,” he told her during one of their late-night phone conversations between Havana and New York in 1964. “No one.” Peter Kornbluh directs the Cuba Documentation Project at the National Security Archive in Washington. He is co-author, with William M. LeoGrande, of Back Channel to Cuba: The Hidden History of Negotiations Between Washington and Havana ."""

In [ ]:
# Get sentiment
lexicon = requests.get("https://www.dropbox.com/s/9f1zh2qxd3l4lj4/AFINN.tsv?raw=1").text.strip().lower()
lexicon = [ w.strip() for w in lexicon.split("\n")[1:] ]
lexicon = { word: int(score) for word, score in map(lambda x: x.split("\t"), lexicon) }

In [ ]:
nltk.download('stopwords')
nltk.download("wordnet")

In [ ]:
# Get stopwords and initialize lemmatizer
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
stop_words[:20]

## Text I

In [ ]:
# Download punctuation data necessary for tokenization
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize

# Tokenize
tokens1 = word_tokenize(text1.lower())
tokens1[:15]
# Lemmatize and remove stop words and punctuation characters
tokens1 = [ lemmatizer.lemmatize(token) for token in tokens1 ]
tokens1 = [ token for token in tokens1 if token not in stop_words and len(token) > 1 ]
tokens1[:15]

The most naive way to compute a sentiment is to just compute the average sentiment score over all words. The question is what to do with words with no score in a lexicon we use? One way is to assign them $0$ scores.

In [ ]:
# Sentiment with assigning 0 scores to words not present in the AFINN lexicon
sent1 = [ lexicon.get(token, 0) for token in tokens1 ]
sent1_score = sum(sent1) / len(sent1)
print("Average sentiment:", sent1_score)

Clearly, this text (according to our method) is rather neutral. We will try to determine whether it is a good description later on.

Perhaps more interesting would be to look at the average over only the words **with** sentiment scores. This would be easier to interpret.
Then, we can also think about a general emotional intensity of a given text defined as a fraction of words with any sentiment among all words.

In [ ]:
sent1_2 = [ lexicon[token] for token in tokens1 if token in lexicon ]
sent1_2_score = sum(sent1_2) / len(sent1_2)
intensity1 = len(sent1_2) / len(tokens1)
print("Average sentiment (only words with scores):", sent1_2_score)
print("Intensity:", intensity1)

We see that according to our simple method the text can be described as somewhat intensive (about 10% of words have sentiment scores), but also rather neutral as negative words are balanced out with positive ones.

# Text II

In [ ]:
# Tokenize
tokens2 = word_tokenize(text2.lower())
tokens2[:15]
# Lemmatize and remove stop words and punctuation characters
tokens2 = [ lemmatizer.lemmatize(token) for token in tokens2 ]
tokens2 = [ token for token in tokens2 if token not in stop_words and len(token) > 1 ]
tokens2[:15]

In [ ]:
# Sentiment with assigning 0 scores to words not present in the AFINN lexicon
sent2 = [ lexicon.get(token, 0) for token in tokens2 ]
sent2_score = sum(sent2) / len(sent2)
print("Average sentiment:", sent2_score)

In this case, the results suggest that the text is more positive than the previous one. To see this better we may look at the ratio of the sentiment scores of the two texts.

In [ ]:
sent2_score / sent1_score

Again, it may be interesting to look at sentiment only over words with sentiment scores as well as the general intensity of the text.

In [ ]:
sent2_2 = [ lexicon[token] for token in tokens2 if token in lexicon ]
sent2_2_score = sum(sent2_2) / len(sent2_2)
intensity2 = len(sent2_2) / len(tokens2)
print("Average sentiment (only words with scores):", sent2_2_score)
print("Intensity:", intensity2)

Here, we see clearly that the text is much more positive, but also slightly less intense.

## Sentiment with VADER

Our previous approach is very simple and could be considered an instance of a bag-of-words type NLP analysis. This means that texts were treated just as an unordered collection of words. We looked just at sentiment scores of individual words and we did not consider any larger context such as grammar, punctuation, etc.

VADER is a slightly more complex approach that takes into account issues such as exclamation marks, negations and adjectival modifiers (i.e. words such as "very"). Its implementation is much more complex than what we did previously and we will not discuss it here. The good thing is that it is implemented in NLTK and extremely easy to use.

In [ ]:
# Import VADER, download its lexicon and initialize an instance of a sentiment analyzer

from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Import sentence tokenizer
from nltk.tokenize import sent_tokenize

nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()

In [ ]:
sentiment1 = vader.polarity_scores(text1)
sentiment2 = vader.polarity_scores(text2)
print(sentiment1, "\n", sentiment2)

In [ ]:
# Sentiment of sentences (more fine-grained picture)
sent_sentiment1 = [ vader.polarity_scores(sent) for sent in sent_tokenize(text1) ]
sent_sentiment1[:5]

In [ ]:
sent_sentiment2 = [ vader.polarity_scores(sent) for sent in sent_tokenize(text2) ]
sent_sentiment2[:5]

In [ ]:
# For instance, we can look at average compound scores over sentences
sent_sentiment1_compound = sum(s['compound'] for s in sent_sentiment1) / len(sent_sentiment1)
sent_sentiment2_compound = sum(s['compound'] for s in sent_sentiment2) / len(sent_sentiment2)
print("Average compound score over sentences (text I):", sent_sentiment1_compound)
print("Average compound score over sentences (text I):", sent_sentiment2_compound)

In [ ]:
# Again, we can look at a ration of what we computed
sent_sentiment2_compound / sent_sentiment1_compound